In [1]:
import os
import sys

import math
from datetime import timezone, datetime

import joblib
import matplotlib.pyplot as plt
import mplfinance as mpf
import numba
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp
import seaborn as sns
import talib

from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score, KFold, TimeSeriesSplit

In [2]:
# TimescaleDB用のユーティリティライブラリの設定
import sys
sys.path.append('..')
from timescaledb_util import TimeScaleDBUtil

_pg_config = {
        'user': os.environ['POSTGRES_USER'],
        'password': os.environ['POSTGRES_PASSWORD'],
        'host': os.environ['POSTGRES_HOST'],
        'port': os.environ['POSTGRES_PORT'],
        'database': os.environ['POSTGRES_DATABASE']
    }
    
_dbutil = TimeScaleDBUtil(user = _pg_config['user'], password = _pg_config['password'], host = _pg_config['host'], port = _pg_config['port'], database = _pg_config['database'])

In [ ]:
_sql = f"SELECT * FROM \"ftx_btc-perp_dollar_cumsum_daily\" ORDER BY time ASC"
df = _dbutil.read_sql_query(sql = _sql)
df = df.set_index("time")    
df

In [ ]:
plt.figure(figsize=(18,6))
ax = plt.subplot(131)
#df['dollar_cumsum'].plot(label='dollar_cumsum', ax=ax)
(df.loc[df.index < '2020-01-01', 'buy_dollar_cumsum'] - df.loc[df.index < '2020-01-01', 'sell_dollar_cumsum']).plot(label='buy_cumsum - sell-cumsum', ax=ax)
ax = plt.subplot(132)
(df.loc[(df.index >= '2020-01-01') & (df.index < '2021-01-01'), 'buy_dollar_cumsum'] - df.loc[(df.index >= '2020-01-01') & (df.index < '2021-01-01'), 'sell_dollar_cumsum']).plot(label='buy_cumsum - sell-cumsum', ax=ax)
ax = plt.subplot(133)
(df.loc[(df.index >= '2021-01-01') & (df.index < '2022-01-01'), 'buy_dollar_cumsum'] - df.loc[(df.index >= '2021-01-01') & (df.index < '2022-01-01'), 'sell_dollar_cumsum']).plot(label='buy_cumsum - sell-cumsum', ax=ax)
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()